# Twitter Data Extraction
Monthly data extraction from Twitter API guided by the following [plan](https://docs.google.com/document/d/1d_4WeDetmZUkk9JJUEWiqqZaBaFsxy1ZDFNiL0JVaok/edit?usp=sharing) | [ISP Selection Guidelines](https://docs.google.com/document/d/12n9hZNdCLmrIVfK05MCa1CUhEYoCR9Ib0fPnxQZql_E/edit?usp=sharing) | [Twitter API Operators](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#limits)

In [5]:
#Import relevant libraries
import tweepy
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta

### 1. SETTING UP & CONNECTING TO THE API

In [6]:
def authenticate():
    
    """
    Function to handle API connection, setup and authentication
    """
    
    #Import the twitter credentials stored in a separate file
    %run ../src/credentials/twitter_credentials
    
    #Create the authentication object
    auth = tweepy.OAuthHandler(api_key,api_secret_key)

    #Set the access token and access token secret
    auth.set_access_token(access_token,access_token_secret)

    #Create the API object
    api = tweepy.API(auth)  
    
    return api

### 2. SPECIFYING VARIABLES FOR THE DATA EXTRACTION

In [7]:
#Dev environment for the full archive endpoint (allows access to all tweets overtime)
dev_env = 'prod'

#THE BELOW SPECIFY HANDLES MANAGED BY THE ISPs or HANDLES THAT APPEAR TO TWEET BRAND PROMOTIONAL CONTENT
#TWEETS FROM THESE HANDLES WILL BE AVOIDED WHEN EXTRACTING
#Note: It is infeasible to address all cases. However, we would expect such tweets to be in the minority

#Spectranet ISP
spectranet_handles = ['-from:spectranet_NG','Spectr_net','SPECTRANETLTE','spectranet__NG']

#IPNX ISP
ipnx_handles = ['-from:ipNXTweet','IpnxSupport','iRecruite']

#Tizeti (Wifi.ng) ISP
tizeti_handles = ['-from:tizeti','wifisupport1']

#Cobranet ISP
cobranet_handles = ['-from:Cobranetisp']

### 2. Extracting the Tweets

#### Splitting the yearly quarters from which data will be extracted into subintervals

In [8]:
def date_range(start, end, intv):
    
    """
    Split Date Range into Roughly Equal Sub Intervals. Adapted from StackOverflow answer by Abhijit(2015)
    Retrieved from https://stackoverflow.com/questions/29721228
    
    Inputs
        - start (str): The start date of the time period
        - end (str): The end date of the time period
        - intv (int): Interval size (i.e. split the duration into roughly 'intv' equal subintervals)
        
    Outputs
        - Generator object containing the subinterval dates
    
    """
    
    #Convert start date to datetime object
    start = datetime.strptime(start,"%Y%m%d")
    
    #Convert end date to datetime object
    end = datetime.strptime(end,"%Y%m%d")
    
    #Find the roughly equal subinterval length
    diff = (end  - start ) / intv
    
    #Compute the subinterval dates and yield as string
    for i in range(intv):
        
        #After the first sub interval, start intervals from the day after the last interval's end day
        if i > 1:
            yield (start + diff * (i-1) + timedelta(1)).strftime("%Y%m%d")
        yield (start + diff * i).strftime("%Y%m%d")
        
    #Compute the last interval
    yield (start + diff * (intv-1) + timedelta(1)).strftime("%Y%m%d")
    yield end.strftime("%Y%m%d")

#### Specifying the query & pulling from the API

In [15]:
def ISP_Tweet_Extractor(api,isp_name, from_date, to_date):
    
    """
    Function to extract tweets for a specified ISP during a specified time frame
    
    Inputs:
     - isp_name (str): Name of the ISP to extract tweets for
     - from_date (str): Earliest date (and time) of posting for any extracted tweet
     - to_date (str): Latest date (and time) of posting for any extracted tweet 
     
    Output:
     - subintv_ISP_tweets (list): List containing API results for yearly quarter subintervals
     
    """
    
    #Connection to api
    api = api
    
    #### ------------------ VARIABLE REFORMATTING ------------------ ####
    #Assign ISP name to variable ensuring it is in lower case
    isp_name = isp_name.lower()
    
    #Reformat the fromDate to YYYYMMDD format
    from_date = from_date.replace('-','')
    
    #Reformat the fromDate to YYYYMMDD format
    to_date = to_date.replace('-','')
    
    #Split the quarter (from_date - to_date) to 5 equal subintervals
    #*100 tweets will be extracted from each subinterval
    intv_dates = [*date_range(from_date, to_date, 5)]
    
    #Get the subinterval date pairs
    date_pairs = [(intv_dates[idx],intv_dates[idx+1]) for idx in range(0,len(intv_dates),2)]
    
    
    #### ------------------ BUILDING THE API QUERY  ------------------ ####
    
    #Join the different handles to form the exclusion portion of the query
    excl_handles = ' -from:'.join(eval(isp_name +'_handles'))
    
    #Query for tweets in Lagos containing the ISP's name and exclude tweets 
    #from the official ISP Twitter handles
    
    #If the ISP is Tizeti, take into account that they are known by multiple names
    if isp_name == 'tizeti':
        
        api_query = f""" tizeti OR wifi.com.ng OR wifi.ng {excl_handles} place:"Lagos,Nigeria" """
    
    else:
        api_query = f""" {isp_name} {excl_handles} place:"Lagos,Nigeria" """
        
        
    #### ------------------ SEARCHING & EXTRACTING THE DATA ------------------ ####
    
    #List to store the subinterval API responses
    subintv_ISP_tweets = []
    
    #For each subinterval
    for start,end in date_pairs:
        
        #Add time to the dates to fit with Twitter API format, 
        start = start + '0000' #midnight
        end = end + '2359' #just before crossing into the next day
        
        #Trying running the query
        try:
            #Full archive search for ISP tweets
            ISP_tweets = api.search_full_archive(dev_env, api_query, fromDate = start, toDate= end)
        
        #If it fails, print the exception raised and the subinterval in question, but continue
        except Exception as e:
            
            print(e,'\n')
            print(f'Subinterval associated with error: [{start},{end}]')
            continue
            
        #Add the subinterval API response to the list
        subintv_ISP_tweets.append(ISP_tweets)
    
    
    return subintv_ISP_tweets


#### Storing the tweets in a pandas dataframe

In [16]:
def tweets_to_df(api_result_list):
    
    """
    Function to extract relevant properties from api results (tweets objects) and store 
    in a pandas dataframe
    
    Input(s):
        - api_result_list (list): List containing API results for a yearly quarter's subintervals
        
    Output(s):
        - main_df (DataFrame): Pandas DataFrame of tweets (and their properties) from the yearly quarter
    
    """
    
    #Empty dataframe to compile data from all yearly quarter subintervals
    main_df = pd.DataFrame()
    
    #Iterate through all the subinterval api results
    for api_result in api_result_list:
        
        #List to store the tweets from each subinterval api_result
        tweets = []
        
        #Iterate through all the tweets
        for tweet in api_result:
            
            #Dictionary to store tweet properties
            tweet_prop = {}
            
            #Store the tweet time
            tweet_prop['Time'] = tweet.created_at
            
            #Store the tweet text – ensuring that the full text is gotten (if truncated)
            if tweet.truncated:
                tweet_prop['Text'] = tweet.extended_tweet['full_text']
            else:
                tweet_prop['Text'] = tweet.text
                
            #Store the coordinates (if available)
            tweet_prop['Coordinates'] = tweet.coordinates
            
            #Store the place
            tweet_prop['Place'] = tweet.place
            
            #Store the source (e.g. Android, iOS, Desktop)
            tweet_prop['Source'] = tweet.source
            
            #Store the tweet in the tweets list
            tweets.append(tweet_prop)
    
        #Convert the dictionary to a pandas dataframe
        df = pd.DataFrame.from_dict(tweets)
        
        #Append the pandas df for the API result to the main df
        main_df = main_df.append(df)
        
    
    return main_df

#### Converting pandas df to csv file

In [17]:
def df_to_csv(df,isp_name,from_date,yearly_quarter):
    
    #Alphanumerics to lowercase
    isp_name = isp_name.lower()
    quarter = yearly_quarter.lower()
    
    #Extract year from date
    year = from_date[:4]
    
    #Convert to CSV to save current tweets obtained from the API
    df.to_csv(f"../data/raw/{isp_name}/{isp_name}_tweets_{quarter}_{year}.csv", index= False)

In [18]:
def main(isp_name=None, from_date=None, to_date=None, yearly_quarter=None, interactive=False):
    
    if interactive:
        #Pass in parameters neeeded for API query
        isp_name = input('ISP Full Name:')
        from_date = input('Start Date (YYYY-MM-DD):')
        to_date = input('End Date (YYYY-MM-DD):')
        yearly_quarter = input('What quarter of the year? (q_):')
        
    else:
        if any(x is None for x in [isp_name,from_date,to_date,yearly_quarter]):
            
            raise ValueError('Please ensure a valid value is passed for all the parameters')
    
    #Connect and authenticate Twitter API
    api = authenticate()
    
    #Pull the data from the API using the query and parameters
    api_results = ISP_Tweet_Extractor(api, isp_name, from_date, to_date)
    
    #Convert the API results into a pandas dataframe
    ISP_tweets = tweets_to_df(api_results)
    
    #Write to csv file
    df_to_csv(ISP_tweets,isp_name,from_date,yearly_quarter)

### Run Code [Option 1]: Pass in arguments in list to run
`Non-interactive` – Uncomment below cell (it executes extraction on running!)

quarterly_dates_2019 = [('2019-01-01','2019-03-31','q1'),('2019-04-01','2019-06-30','q2'),
                        ('2019-07-01','2019-09-30','q3'),('2019-10-01','2019-12-31','q4')]

quarterly_dates_2020 = [('2020-01-01','2020-03-31','q1'),('2020-04-01','2020-06-30','q2'),
                        ('2020-07-01','2020-09-30','q3'),('2020-10-01','2020-12-31','q4')]

for start, end, quarter in quarterly_dates_2020:
    
    if __name__ == "__main__":
        main('spectranet',start,end,quarter)
    time.sleep(1)


### Run Code [Option 2]: Pass in arguments one after the other
`Interactive`

In [14]:
if __name__ == "__main__":
    main(interactive=True)

ISP Full Name: tizeti
Start Date (YYYY-MM-DD): 2019-01-01
End Date (YYYY-MM-DD): 2019-03-31
What quarter of the year? (q_): q1


422 Unprocessable Entity
There were errors processing your request: Reference to invalid operator 'is:retweet'. Operator is not available in current product or product packaging. Please refer to complete available operator list at http://t.co/operators. (at position 92) 

Subinterval associated with error: [201901010000,201901182359]
422 Unprocessable Entity
There were errors processing your request: Reference to invalid operator 'is:retweet'. Operator is not available in current product or product packaging. Please refer to complete available operator list at http://t.co/operators. (at position 92) 

Subinterval associated with error: [201901190000,201902052359]
422 Unprocessable Entity
There were errors processing your request: Reference to invalid operator 'is:retweet'. Operator is not available in current product or product packaging. Please refer to complete available operator list at http://t.co/operators. (at position 92) 

Subinterval associated with error: [201902060000,201902